In [1]:
import json
import sys
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
FILEPATHS = {
    "NER": {
        "train": "sciner_models/sciner-scibert/train_ner.json",
        "val": "sciner_models/sciner-scibert/val_ner.json",
    },
    "RE": {
        "train": "scire_models/scire-scibert/train_re.json",
        "val": "scire_models/scire-scibert/val_re.json",
    }
}

In [3]:
def print_progress(curr, full, prefix="", bar_size=30):    
    bar = int((curr+1)/full*bar_size)
    sys.stdout.write(f"\r{prefix}[{'='*bar}{' '*(bar_size-bar)}] {curr+1}/{full}")
    sys.stdout.flush()
    

In [4]:
filepath = FILEPATHS["NER"]["val"]
with open(filepath, 'r') as json_file:
    json_list = list(json_file)
data_ner = [json.loads(json_str) for json_str in json_list]
print(data_ner[0])

{'doc_key': 'SP:7f4b788b00a2a10bcd60351c3e04c8f597101e96', 'sentences': [['This', 'paper', 'studies', 'FL', 'under', 'local', 'differential', 'privacy', 'constraints', '.'], ['They', 'identify', 'two', 'major', 'concerns', 'in', 'designing', 'practical', 'privacy-preserving', 'FL', 'algorithms', ':', 'communication', 'efficiency', 'and', 'high\x02dimensional', 'compatibility', ',', 'and', 'develop', 'a', 'gradient-based', 'learning', 'algorithm', 'sqSGD', 'that', 'addresses', 'both', 'concerns', '.'], ['They', 'improve', 'the', 'base', 'algorithm', 'in', 'two', 'ways', ':', 'First', ',', 'apply', 'a', 'gradient', 'subsampling', 'strategy', 'that', 'offers', 'simultaneously', 'better', 'training', 'performance', 'and', 'smaller', 'communication', 'costs', '.'], ['Secondly', ',', 'utilize', 'randomized', 'rotation', 'as', 'a', 'preprocessing', 'step', 'to', 'reduce', 'quantization', 'error', '.']], 'ner': [[], [], [], []], 'relations': [[], [], [], []], 'predicted_ner': [[[3, 3, 'Task'],

In [5]:
result = data_ner[0]
all_sentences = [j for i in result["sentences"] for j in i]

for idx, sent in enumerate(result["sentences"]):
    print(f"Sentence {idx}: ")
    print(" ".join(sent))
    for pred_ner in result["predicted_ner"][idx]:
        start_idx = pred_ner[0]
        end_idx   = pred_ner[1]+1
        ent       = " ".join(all_sentences[start_idx:end_idx])
        print("\t{: >20}: {: <50}".format(pred_ner[2], ent))
    print()

Sentence 0: 
This paper studies FL under local differential privacy constraints .
	                Task: FL                                                
	 OtherScientificTerm: local differential privacy constraints            

Sentence 1: 
They identify two major concerns in designing practical privacy-preserving FL algorithms : communication efficiency and highdimensional compatibility , and develop a gradient-based learning algorithm sqSGD that addresses both concerns .
	              Method: privacy-preserving FL algorithms                  
	              Metric: communication efficiency                          
	              Metric: highdimensional compatibility                    
	              Method: gradient-based learning algorithm sqSGD           

Sentence 2: 
They improve the base algorithm in two ways : First , apply a gradient subsampling strategy that offers simultaneously better training performance and smaller communication costs .
	             Generic: base

In [6]:
filepath = FILEPATHS["RE"]["val"]
with open(filepath, 'r') as json_file:
    json_list = list(json_file)
data_len = len(json_list)
data_re = [json.loads(json_str) for json_str in json_list]

In [7]:
result = data_re[2]
print(result)

{'doc_key': 'SP:7f4b788b00a2a10bcd60351c3e04c8f597101e96', 'sentences': [['The', 'paper', 'proposed', 'a', 'differentially', 'private', 'training', 'algorithm', 'for', 'federated', 'learning', '.'], ['The', 'target', 'is', 'to', 'achieve', 'communication', 'reduction', 'while', 'keeping', 'differential', 'privacy', 'during', 'training', '.'], ['The', 'proposed', 'algorithm', 'adds', 'a', 'few', 'new', 'components', 'to', 'SGD', ',', 'including', 'a', 'privacy', 'mechanism', ',', 'a', 'random', 'rotation', 'to', 'reduce', 'quantization', 'error', ',', 'a', 'gradient', 'coordinate', 'selection', 'mechanism', 'to', 'reduce', 'communication/computation', '.'], ['Experiments', 'with', 'high', '\\epsilon', 'local', 'differentially', 'privacy', 'guarantees', 'are', 'conducted', '.'], ['The', 'proposed', 'algorithm', 'outperforms', 'a', 'baseline', 'algorithm', '.']], 'ner': [[], [], [], [], []], 'relations': [[], [], [], [], []], 'predicted_ner': [[[4, 7, 'Method'], [9, 10, 'Method']], [[17, 

In [8]:
all_sentences = [j for i in result["sentences"] for j in i]
for idx, sent in enumerate(result["sentences"]):
    print(f"Sentence {idx}: ")
    print(" ".join(sent))
    print(f"[Entity type]")
    for pred_ner in result["predicted_ner"][idx]:
        start_idx = pred_ner[0]
        end_idx   = pred_ner[1]+1
        ent       = " ".join(all_sentences[start_idx:end_idx])
        print("\t{: >20}: {: <50}".format(pred_ner[2], ent))
    print(f"[Triples]")
    for re in result["predicted_re"][idx][1]:
        s = " ".join(all_sentences[re[0][0]:(re[0][1]+1)])
        p = re[2]
        o = " ".join(all_sentences[re[1][0]:(re[1][1]+1)])
        print(f"({s}) ({p}) ({o})")
    
    print()

Sentence 0: 
The paper proposed a differentially private training algorithm for federated learning .
[Entity type]
	              Method: differentially private training algorithm         
	              Method: federated learning                                
[Triples]
(differentially private training algorithm) (USED-FOR) (federated learning)

Sentence 1: 
The target is to achieve communication reduction while keeping differential privacy during training .
[Entity type]
	                Task: communication reduction                           
	 OtherScientificTerm: differential privacy                              
[Triples]

Sentence 2: 
The proposed algorithm adds a few new components to SGD , including a privacy mechanism , a random rotation to reduce quantization error , a gradient coordinate selection mechanism to reduce communication/computation .
[Entity type]
	             Generic: algorithm                                         
	             Generic: components         

In [9]:
def display_triple(sent, rel=""):
    # s_inx_beg = rel[0][0]
    # s_inx_end = rel[0][1]+1
    # o_inx_beg = rel[1][0]
    # o_inx_end = rel[1][1]+1
    # s = " ".join(all_sentences[s_inx_beg:s_inx_end])
    # p = re[2]
    # s = " ".join(all_sentences[o_inx_beg:o_inx_end])
    for w in sent:
        print(f"{w}", end=" ")

In [10]:
display_triple(all_sentences)

The paper proposed a differentially private training algorithm for federated learning . The target is to achieve communication reduction while keeping differential privacy during training . The proposed algorithm adds a few new components to SGD , including a privacy mechanism , a random rotation to reduce quantization error , a gradient coordinate selection mechanism to reduce communication/computation . Experiments with high \epsilon local differentially privacy guarantees are conducted . The proposed algorithm outperforms a baseline algorithm . 

In [21]:
import spacy
from spacy import displacy
from spacy.tokens import Span
colors = {
    'Task':     "#DE3163", 
    "Method":   "#6495ED", 
    "Metric":   "#FF00FF", 
    "Material": "#40E0D0", 
    "Generic":  "#FFBF00",
    "OtherScientificTerm": "#9FE2BF", 
}
options = {"ents": colors.keys(), "colors": colors, "compact": True,}
nlp = spacy.load("en_core_web_sm")

def display_ent(doc, ents):
    doc = nlp(doc)
    span_list = [Span(doc, start, end, ent_type) for start, end, ent_type in ents]
    doc.set_ents(span_list)
    ents = list(doc.ents)
    displacy.render(doc, style="ent", options=options)

In [22]:
def display_graph(doc, rels):
    pass

In [23]:
flatten_ner = [j for i in result["predicted_ner"] for j in i]
flatten_re  = [j for i in result["predicted_re"] for j in i[1]]
for i in result["predicted_re"]:
    print(i)

[0, [[[4, 7], [9, 10], 'USED-FOR']]]
[1, []]
[2, [[[39, 40], [43, 44], 'CONJUNCTION'], [[43, 44], [47, 48], 'USED-FOR'], [[43, 44], [51, 54], 'CONJUNCTION'], [[51, 54], [57, 57], 'USED-FOR'], [[28, 28], [35, 35], 'USED-FOR'], [[33, 33], [35, 35], 'USED-FOR'], [[33, 33], [28, 28], 'PART-OF'], [[43, 44], [33, 33], 'PART-OF'], [[51, 54], [33, 33], 'PART-OF'], [[39, 40], [33, 33], 'PART-OF']]]
[3, []]
[4, [[[72, 72], [75, 76], 'COMPARE']]]


In [24]:
display_ent(" ".join(all_sentences), flatten_ner)

In [25]:
from graphviz import Digraph
# Create Digraph object
dot = Digraph()

In [26]:
# Add nodes 1 and 2
dot.node('1')
dot.node('2')

# Add edge between 1 and 2
dot.edges(['12'])



In [ ]:
from IPython.display import Image
Image('digraph.png')

In [33]:
import graphviz

f = graphviz.Digraph('finite_state_machine', filename='fsm.gv')
f.attr(rankdir='LR', size='8,5')

f.attr('node', shape='doublecircle')
f.node('LR_0')
f.node('LR_3')
f.node('LR_4')
f.node('LR_8')

f.attr('node', shape='circle')
f.edge('LR_0', 'LR_2', label='SS(B)')
f.edge('LR_0', 'LR_1', label='SS(S)')
f.edge('LR_1', 'LR_3', label='S($end)')
f.edge('LR_2', 'LR_6', label='SS(b)')
f.edge('LR_2', 'LR_5', label='SS(a)')
f.edge('LR_2', 'LR_4', label='S(A)')
f.edge('LR_5', 'LR_7', label='S(b)')
f.edge('LR_5', 'LR_5', label='S(a)')
f.edge('LR_6', 'LR_6', label='S(b)')
f.edge('LR_6', 'LR_5', label='S(a)')
f.edge('LR_7', 'LR_8', label='S(b)')
f.edge('LR_7', 'LR_5', label='S(a)')
f.edge('LR_8', 'LR_6', label='S(b)')
f.edge('LR_8', 'LR_5', label='S(a)')



In [34]:
f.render("graphname", format="png")

ExecutableNotFound: failed to execute PosixPath('dot'), make sure the Graphviz executables are on your systems' PATH